## Preprocessing and Normalization

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import seaborn as sns
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from random import sample
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
import numpy as np
import warnings

warnings.filterwarnings("ignore")
get_ipython().run_line_magic('matplotlib', 'inline')

# Create dataframes for red wine quality dataset
df_redWine = pd.read_csv("winequality-red.csv")

nancol = []
no_ph = None 

i = 0
while i < len(df_redWine):
  if df_redWine.iat[i,11] == 3:
    df_redWine.iat[i,11] = 4
  elif df_redWine.iat[i,11] == 8:
    df_redWine.iat[i,11] == 7
  elif df_redWine.iat[i,11] == 6:
    df_redWine.iat[i,11] = 5
  i = i + 1

df_wine = df_redWine.drop_duplicates().dropna() #duplicate rows and rows with null values are dropped
df_wine_copy = df_wine.copy() # copy of the dataframe is made

#min-max normalization
for column in df_wine_copy.columns:
  df_wine_copy[column] = (df_wine_copy[column] - df_wine_copy[column].min()) / (df_wine_copy[column].max() - df_wine_copy[column].min())

df_wine_copy.loc[:, ['quality']] = df_wine[['quality']] #overwrite quality column with the original quality column values
df_wine = df_wine_copy # set the original dataframe to the copy 

y = df_wine['quality']
x = df_wine.drop(columns=['quality'])

oversample = RandomOverSampler(sampling_strategy='minority')
x, y = oversample.fit_resample(x, y)

x['quality'] = y

y = df_wine['quality']
x = df_wine.drop(columns=['quality'])

# stratified K Fold for 10 splits
skf = StratifiedKFold(n_splits=10)

fold = 1 # The current fold

# Split the data 10 times for 10 folds
# Call each classifier method for each fold
# Confusion Matrices and accuracies of each classifier are outputted for each fold
for train_index, test_index in skf.split(x, y):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    print('Fold: ', fold)
    print()
    customNB(x_train, x_test, y_train, y_test)
    complementNB(x_train, x_test, y_train, y_test)
    gaussianNB(x_train, x_test, y_train, y_test)
    decTree(x_train, x_test, y_train, y_test)
    svm(x_train, x_test, y_train, y_test)
    logisticRegression(x_train, x_test, y_train, y_test)
    kMeans(x_train, x_test, y_train, y_test)
    kNN(x_train, x_test, y_train, y_test)
    print('\n---------------------\n')
    fold = fold + 1


## Gaussian and Complement Naive Bayes


In [ ]:
  def complementNB(x_train, x_test, y_train, y_test):
  # Complement Naive Bayes
  cnbClassifier = ComplementNB()
  cnbClassifier.fit(x_train, y_train)

  prediction = cnbClassifier.predict(x_test)
  print(f"Complement Naive Bayes Accuracy : {accuracy_score(y_test, prediction) * 100} % \n\n")

def gaussianNB(x_train, x_test, y_train, y_test):
  # Gaussian Naive Bayes
  gnbClassifier = GaussianNB()
  gnbClassifier.fit(x_train, y_train)

  gaussianPrediction = gnbClassifier.predict(x_test)
  print(f"Gaussian Naive Bayes Accuracy : {accuracy_score(y_test, gaussianPrediction) * 100} % \n\n")

## Decision Tree


In [ ]:
def decTree(x_train, x_test, y_train, y_test):
  # Decision Tree 
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(x_train, y_train)
  pred = clf.predict(x_test)
  print(f"Decision Tree Accuracy : {accuracy_score(y_test, pred) * 100} % \n\n")

  # Decision Tree with ensemble learning
  bagClassifier = BaggingClassifier(base_estimator=clf, n_estimators=20, max_samples=.5)
  bagClassifier.fit(x_train, y_train)
  print(f"Decision Tree Bagging Classifier Accuracy: {bagClassifier.score(x_test, y_test) * 100} % \n\n")

## Support Vector Machine

In [ ]:
def buildSVM(Xtrain, Xtest, ytrain, ytest, message):
    classifier = SVC(random_state=5, kernel="rbf", C=10, gamma=0.1)
    trained_model = classifier.fit(Xtrain, ytrain)
    y_pred = classifier.predict(Xtest)
    print(message, accuracy_score(ytest, y_pred) * 100, "\n\n")

In [ ]:
def svm(x_train, x_test, y_train, y_test):  
  global nancol
  global no_ph
  tuned_parameters = [
      {
          'kernel': ['rbf'],
          'gamma': [1e-1, 1e-2, 1e-3, 1e-4],
          'C': [1, 10, 100, 1000]
      },
      {
          'kernel': ['linear'],
          'gamma': [1e-1, 1e-2, 1e-3, 1e-4],
          'C': [1, 10, 100, 1000]
      },
      {
          'kernel': ['sigmoid'],
          'gamma': [1e-1, 1e-2, 1e-3, 1e-4],
          'C': [1, 10, 100, 1000]
      },
      # {
      #     'kernel': ['poly'],
      #     'gamma': [1e-1, 1e-2, 1e-3, 1e-4],
      #     'C': [1, 10, 100, 1000],
      #     'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
      # }
  ]

  # Support Vector Machine
  sc = StandardScaler()
  X_train = sc.fit_transform(x_train)
  X_test = sc.transform(x_test)
  classifier = SVC(random_state=5, kernel="rbf", C=10, gamma=0.1)
  trained_model = classifier.fit(x_train, y_train)
  y_pred = classifier.predict(x_test)
  print(f"Support Vector Machine Accuracy : {accuracy_score(y_test, y_pred) * 100} % \n\n")

  erased = len(X_train) * 33 // 100
  not_erased = len(X_train) - erased
  nancol = sample(range(len(X_train)), erased) #παίρνω τυχαίο δείγμα του 33% των rows
  for index in nancol:
      X_train[index][8] = None

  no_ph = np.delete(X_train,[8], axis=1) 
  no_ph_test = np.delete(X_test,[8], axis=1)
  buildSVM(no_ph,no_ph_test,y_train,y_test, "SVM Accuracy (No pH): ")

  median = 0
  for index in range(len(X_train)):
      if index not in nancol:
          median += X_train[index][8]

  X_median = np.array(X_train)
  median = median / not_erased
  for index in range(len(X_median)):
      if index in nancol:
        X_median[index][8] = median

  buildSVM(X_median, X_test, y_train, y_test, "SVM Accuracy (pH with median values): ")

## Logistic Regression

In [ ]:
def logisticRegression(x_train, x_test, y_train, y_test):  
  global nancol
  y_ph = x_train.loc[:, ['pH']] 
  
  sc = StandardScaler()
  xg_train = []
  yg_train = []
  xg_test = []
  yg_test = []
  X_train2 = x_train.values
  np.set_printoptions(suppress=True)

  X_train2[:,0] = X_train2[:,0]*10
  X_train2[:,1] = X_train2[:,1]*1000
  X_train2[:,2] = X_train2[:,2]*100
  X_train2[:,3] = X_train2[:,3]*10
  X_train2[:,4] = X_train2[:,4]*1000
  X_train2[:,7] = X_train2[:,7]*100000
  X_train2[:,8] = X_train2[:,8]*100
  X_train2[:,9] = X_train2[:,9]*100
  X_train2[:,10] = X_train2[:,10]*10
  X_train2.astype(int)

  no_ph = np.delete(X_train2,[8], axis=1) 
  for i in range(len(X_train2)):
      if i not in nancol:
          xg_train.append(no_ph[i])
          yg_train.append(y_train.iat[i])
      if i in nancol:
          xg_test.append(no_ph[i])
          yg_test.append(y_train.iat[i])

  logistic_regression = LogisticRegression(max_iter=120000)
  logistic_regression.fit(xg_train, yg_train)
  y_pred = logistic_regression.predict(xg_test)

  j=0
  for i in range(len(X_train2)):
      if i in nancol:
        X_train2[i][8] = y_pred[j]
        j += 1

  X_train2[:,0] = X_train2[:,0]/10
  X_train2[:,1] = X_train2[:,1]/1000
  X_train2[:,2] = X_train2[:,2]/100
  X_train2[:,3] = X_train2[:,3]/10
  X_train2[:,4] = X_train2[:,4]/1000
  X_train2[:,7] = X_train2[:,7]/100000
  X_train2[:,8] = X_train2[:,8]/100
  X_train2[:,9] = X_train2[:,9]/100
  X_train2[:,10] = X_train2[:,10]/10
  X_train2 = sc.fit_transform(X_train2)

  buildSVM(X_train2, x_test, y_train, y_test, "Logistic Regression Accuracy: ")

## K-Means

In [ ]:
def kMeans(x_train, x_test, y_train, y_test):  
  global no_ph
  no_ph1 = x_train.drop('pH', axis=1)
  X_kmeans = x_train.values.tolist()
  X_kplot = no_ph1.values.tolist()
  
  pH=8
  for index in nancol:
      X_kmeans[index][pH] = None

  wcss = []
  for i in range(1, 11):
      kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
      kmeans.fit(X_kplot)
      wcss.append(kmeans.inertia_)
  plt.plot(range(1, 11), wcss)
  plt.title('Elbow Method')
  plt.xlabel('Number of clusters')
  plt.ylabel('WCSS')
  plt.savefig("kmeans.png")
  plt.show()
  #Αυτα ειναι για να δειξουμε γιατι 4


  kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)
  centers = kmeans.fit(X_kplot).cluster_centers_

  clusters = []
  for i in range(len(centers)):
      clusters.append([])
  for i in range(len(X_kplot)):
      dist = []
      for j in range(len(centers)):
          dist.append([np.linalg.norm(X_kplot[i]-centers[j]),j])
      dist.sort()
      clusters[dist[0][1]].append(i)

  median = []
  for i in range(len(clusters)):
      median.append(0)

  clust_num = 0
  #print(len(clusters))
  for cluster in clusters:
      not_erased=0
      for index in cluster:
          if X_kmeans[index][pH] is not None:
              median[clust_num] += X_kmeans[index][pH]
              not_erased+=1
      if not_erased != 0:
          median[clust_num] /=not_erased
      for index in cluster:
          if X_kmeans[index][pH] is None:
              X_kmeans[index][pH] = median[clust_num]
      clust_num+=1

  sc = StandardScaler()
  X_noph = sc.fit_transform(no_ph)
  X_kmeans = sc.fit_transform(X_kmeans)
  print()
  buildSVM(X_kmeans, x_test, y_train, y_test, "K-Means Accuracy: ")

## K-Nearest Neighbors


In [ ]:
def run_model(model, k, x_train, x_test, y_train, y_test):
  model.fit(x_train, y_train)
  preds = model.predict(x_test)
  print("K = ", k, " Accuracy Score is: ", accuracy_score(y_test,preds) *100)

def kNN(x_train, x_test, y_train, y_test):
  kvalues = [3, 5, 15, 18, 20, 25]
  print("K-Nearest Neighbors Accuracies")
  for k in kvalues:
    model = KNeighborsClassifier(n_neighbors= k)
    run_model(model, k, x_train, x_test, y_train, y_test)


## Custom Naive Bayes

In [ ]:
def customNB(x_train, x_test, y_train, y_test):
  quantity4 = 0
  quantity5 = 0
  quantity7 = 0

  quality4means = []
  quality5means = []
  quality7means = []

  quality4stds = []
  quality5stds = []
  quality7stds = []

  i = 0
  while i < len(y_train):
    if y_train.iat[i] == 4:
      quantity4 = quantity4 + 1
    elif y_train.iat[i] == 5:
      quantity5 = quantity5 + 1
    elif y_train.iat[i] == 7:
      quantity7 = quantity7 + 1
    i = i + 1

  priorProb4 = quantity4 / len(y_train)
  priorProb5 = quantity5 / len(y_train)
  priorProb7 = quantity7 / len(y_train)

  c = 0
  while c <= 10:
    r = 0
    mean4 = 0
    mean5 = 0
    mean7 = 0
    
    while r < len(x_train):
      if y_train.iat[r] == 4:
        mean4 = mean4 + x_train.iat[r, c]
      elif y_train.iat[r] == 5:
        mean5 = mean5 + x_train.iat[r, c]
      elif y_train.iat[r] == 7:
        mean7 = mean7 + x_train.iat[r, c]
      r = r + 1

    if quantity4 != 0:
      mean4 = mean4 / quantity4
      quality4means.append(mean4)
    else:
      quality4means.append(0)
    if quantity5 != 0:
      mean5 = mean5 / quantity5
      quality5means.append(mean5)
    else:
      quality5means.append(0)
    if quantity7 != 0:
      mean7 = mean7 / quantity7
      quality7means.append(mean7)
    else:
      quality7means.append(0)
    c = c + 1
  
  c = 0
  while c <= 10:
    r = 0
    std4 = 0
    std5 = 0
    std7 = 0
    while r < len(x_train):
      if y_train.iat[r] == 4:
        std4 = std4 + ((x_train.iat[r, c] - quality4means[c])**2)
      elif y_train.iat[r] == 5:
        std5 = std5 + ((x_train.iat[r, c] - quality5means[c])**2)
      elif y_train.iat[r] == 7:
        std7 = std7 + ((x_train.iat[r, c] - quality7means[c])**2)
      r = r + 1

    if quantity4 != 0:
      std4 = math.sqrt(std4 / quantity4)
      quality4stds.append(std4)
    else:
      quality4stds.append(0)
    if quantity5 != 0:
      std5 = math.sqrt(std5 / quantity5)
      quality5stds.append(std5)
    else:
      quality5stds.append(0)
    if quantity7 != 0:
      std7 = math.sqrt(std7 / quantity7)
      quality7stds.append(std7)
    else:
      quality7stds.append(0)
    c = c + 1

  predictedClasses = []

  r = 0
  while r < len(x_test):
    c = 0
    prob4 = 1
    prob5 = 1
    prob7 = 1

    while c <= 10:
      prob4 = prob4 * norm.pdf(x_test.iat[r, c], quality4means[c], quality4stds[c])
      prob5 = prob5 * norm.pdf(x_test.iat[r, c], quality5means[c], quality5stds[c])
      prob7 = prob7 * norm.pdf(x_test.iat[r, c], quality7means[c], quality7stds[c])
      c = c + 1
    
    prob4 = prob4 * priorProb4
    prob5 = prob5 * priorProb5
    prob7 = prob7 * priorProb7

    probList = [prob4, prob5, prob7]
    maxProb = np.nanmax(probList)
    maxProbIndex = probList.index(maxProb)
    
    if maxProbIndex == 0:
      predictedClasses.append(4)
    elif maxProbIndex == 1:
      predictedClasses.append(5)
    elif maxProbIndex == 2:
      predictedClasses.append(7)
    r = r + 1

  r = 0
  correct = 0
  while r < len(x_test):
    if y_test.iat[r] == predictedClasses[r]:
      correct = correct + 1
    r = r + 1
  accuracy = correct / len(x_test)
  print("Custom Naive Bayes Accuracy : ", accuracy * 100,"% \n\n")